# Background


PCA is a commonly used dimensionality reduction technique that seeks to maximize the sample variance of the centered and normalized data according to (formula from pg 23 of 2.6.2.5 in latex) where $X$ denotes the data matrix, and $\phi_1$ is the vector of loadings for the first principal component. A mathematically equivalent formulation is given by the eigenvectors of the covariance matrix $X^TX$, with the eigenvector associated with the largest eigenvalue corresponding to the first principal component. The process projects the data onto a lower dimension linear subspace, such that the data maintains as much of its original information as possible.


An issue with utilizing standard PCA arises when considering linear separability. Singular value decomposition is a linear transformation, and as such, PCA does not work well when subjected to data with nonlinear decision boundaries.


Kernel PCA is actually a more general case of standard PCA, and can generally handle nonlinear cases. The idea is to implement a nonlinear kernel function to project the data into a larger dimensional feature space where the data is then linearly separable, and then perform PCA. (figure 12.16 pg 587 microsoft).


The MNIST dataset, as we have previously discussed, is a widely utilized set for training machine learning models. The set of handwritten digits consists of 60,000 training images, and 10,000 testing images, all of which are normalized according to size and grayscale levels. In this set in particular, the data is confined to a nonlinear subspace.

# Theory

First, we assume we have a nonlinear transformation $\phi(x): D \rightarrow M$ with $\lvert D\rvert \gg \lvert M\rvert$. Each vector in the entry matrix $A$ is then projected using $\phi(x_i)$ for all $x_i \in A$.

Standard PCA can be used in this new feature space, but it is costly and inefficient. For example the dot product of two vectors $x$ and $y$ calculated in the feature space
$\phi(x)\cdot\phi(y)$ can be written in the kernel representation as
$$
k(x,y)=\phi(x)\cdot\phi(y)
$$
which allows calcuation in $M$ without having to carry out the map $\phi$. If $M$ is high-dimensional, a closed form expression of $k$ would greatly reduce the computational cost of the procedure. Naturally, the choice of kernel is dictated by the problem at hand. A specific example cited by Scholkopf et al. (1998) involves PCA on pixel images in which the kernel
$$
\begin{align*}
k(x,y)&=(x\cdot y)^d=\left(\sum_{j=1}^Nx_j\cdot y_j\right)\\
&=\sum_{j_1,\cdots,j_d=1}^Nx_{j_1}\cdot\cdots x_{j_d}\cdot y_{j_1}\cdot\cdots y_{j_d}
\end{align*}
$$
corresponds to a dot product in the space of $d$th order monomials of the input coordinates where $N$ is the dimensionality of $D$. The dimensionality of $M$ is then $\frac{(N+d-1)!}{d!(N-1)!}$ which scales as $N^d$. Therefore, pedestrian a $16\times16$ pixel image with $d=5$ brings the dimensionality of $M$ to $10^{10}$ which is clearly far too large to reasonably compute in so using kernels is the only way to find any higher order patterns.

Because we do not have a good guess as to what order patters will emeerge from our problem, we follow the convention of using the Gaussian kernel. 

We define our kernel method as:

$$
\kappa(x_i, x_j) = \exp\left[-\frac{\lVert x - y \rVert^2}{2\sigma^2}\right] = \phi(x_i)^T\phi(x_j)
$$

Generally speaking, we cannot assume that the projected features have zero mean. Thus we have to centralize the data:

$$
\widetilde\phi(x_n) = \phi(x_n) - \frac{1}{N}\sum_{i=1}^{N}\phi(x_i)
$$

The elements of the Gram matrix are then

$$
\begin{align*}
    \widetilde{K}_{nm} & = \widetilde\phi(x_n)^T\widetilde\phi(x_m)\\
        & = \phi(x_n)^T\phi(x_m) - \frac{1}{N}\sum_{i=1}^{N}\phi(x_n)^T\phi(x_i) - \frac{1}{N}\sum_{i=1}^{N}\phi(x_i)^T\phi(x_m) + \frac{1}{N^2}\sum\limits_{j=1}^N\sum\limits_{i=1}^N \phi(x_j)^T\phi(x_i)\\
        & = k(x_n,x_m) - \frac{1}{N}\sum_{i=1}^{N} k(x_i,x_m) - \frac{1}{N}\sum_{i=1}^{N} k(x_n,x_i) + \frac{1}{N^2}\sum\limits_{j=1}^N\sum\limits_{i=1}^N k(x_j,x_i)\\
\end{align*}
$$

Rewriting in matrix notation yields

$$
\widetilde{\bf K} = {\bf K - 1_{\text{N}}K - K1_{\text{N}} + 1_{\text{N}}K1_{\text{N}}}
$$

Where, by convention, ${\bf 1_{\text{N}}}$ is the $N \times N$ matrix with $1/N$ as entries. Using $\widetilde{\bf K}$ in place of ${\bf K}$ will ensure the projected features have zero mean. Now, we construct the $M$ x $M$ covarience matrix of the projected features using

$$
C = \frac{1}{N}\sum_{i=1}^{N}\phi(x_i)\phi(x_i)^T
$$

The eigenvalues and eigenvectors of $C$ are given by

$$
Cv_k = \lambda_{k}v_k \space\space\space \forall \space 1 \leq k \leq M
$$

It follows that

$$
\left(\frac{1}{N}\sum_{i=1}^{N}\phi(x_i)\phi(x_i)^T\right)v_k = \lambda_{k}v_k
$$

Let $a_k$ be a principal component in $\mathbb{R}^d$, then $v_k = \sum_{i=1}^{N}a_{k,i}\phi(x_i)$. Substituting this into the previous equation gives

$$
\left(\frac{1}{N}\sum_{i=1}^{N}\phi(x_i)\phi(x_i)^T\right)\sum_{j=1}^{N}a_{k,j}\phi(x_j) = \lambda_{k}\sum_{j=1}^{N}a_{k,j}\phi(x_j)
$$

Now we define the kernel function as $\kappa(x_i, x_j) = \phi(x_i)^T\phi(x_j)$. If we multiply the eigenvector equation on both sides by $\phi(x_l)^T$, then we get

$$
\begin{align*}
    \frac{1}{N}\sum_{i=1}^{N}\phi(x_i)^T\phi(x_i)\phi(x_i)^T\sum_{j=1}^{N}a_{k,j}\phi(x_j) = \lambda_{k}\sum_{j=1}^{N}a_{k,j}\phi(x_l)^T\phi(x_j) & \implies \sum_{i=1}^{N}\kappa(x_l, x_i)\sum_{j=1}^{N}a_{k,j}\kappa(x_i, x_j) = N\lambda_{k}\sum_{j=1}^{N}a_{k,j}\kappa(x_l, x_j)\\
    & \implies K^2a_k = N\lambda_{k}Ka_k\\
    & \implies Ka_k = N\lambda_{k}a_k
\end{align*}
$$

Thus, the kernel principal components are found using:

$$
y_k(x) = \phi(x)^Tv_k = \sum_{j=1}^{N}a_{k,j}\kappa(x, x_j)
$$

# Implementation

Before we start implementing kernel PCA in Python, we need to import the neccessary libraries and set up Google Colab

In [ ]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

Kernel PCA consists of four steps:
1. Constructing the kernel matrix $K$ from the training data set
2. Centralizing $K$ by computing the Gram matrix $\widetilde{K}$
3. Solving for the vectors $a_i$
4. Computing the kernel principal components $y_k(x)$

Step 1: We create a function that takes in a training data set and sigma value and constructs the Gaussian kernel.

In [ ]:
# Create Gaussian Kernel
def gaus_kernel(X, sigma):
	row, col = X.shape

	K = np.zeros([row, row])

	for i in range(0,row):
		for j in range(0,row):
			v_i = X[i]
			v_j = X[j]
			K[i,j] = np.exp(-np.linalg.norm(v_i.T - v_j.T)**2/(2 * np.power(sigma, 2)))

	return K

Step 2: We create a function to compute the Gram matrix to ensure the projected features are centralized.

In [ ]:
# Construct the Gram Matrix
def gram_matrix(K):
	row, col = K.shape

	one_N = np.ones((row, col)) / row
	K_tilde = K - (one_N @ K) - (K @ one_N) + (one_N @ K @ one_N)
	return K_tilde

Step 3: Now that we have the Gram matrix, we need to solve for the vectors $a_i$

In [ ]:
# Solve for the eigenvectors a_i
def solve_ai(K_tilde):
	eig_vals, eig_vectors = np.linalg.eigh(K_tilde)
	eig_vals, eig_vectors = eig_vals[::-1], eig_vectors[:, ::-1]
	return eig_vectors


Step 4: Compute the kernel principal components $y_k(x)$

In [ ]:
# Compute the kernel principal components
def compute_kernel_components(eigvecs):
    y = []
    for a_k in eigvecs:
        y_k = np.zeros((len(a_k), 1))
        for x in X:
            for i in range(0, len(X)):
                k = np.exp(-np.linalg.norm(x.T - x[i].T)**2/(2 * np.power(sigma, 2)))
                y_k += a_k[i] * k
            y.append(y_k)
    return y

# Data

Now that we have created functions for the four steps of kernel PCA, we can apply them to the MNIST dataset, but first we must get the training data set and filter it to be only zeros and ones.

In [ ]:
mnist = keras.datasets.mnist
(train_imgs, train_labels), (test_imgs, test_labels) = mnist.load_data()

i01 = [i for i in range(len(train_labels[0:100])) if (train_labels[i]==0) or (train_labels[i]==1)]
imgs01 = train_imgs[i01]
labels01 = train_labels[i01]

Now, we can perform kernel PCA on the training data set

In [ ]:
X = np.vstack([imgs01[i].flatten() for i in range(len(labels01))])
y = labels01

K = gaus_kernel(X, 20)
K_tilde = gram_matrix(K)
eigvecs = solve_ai(K_tilde)
K_kpca = compute_kernel_components(eigvecs)

df = pd.DataFrame()
df["y"] = y
df["comp-1"] = X_kpca[:,0]
df["comp-2"] = X_kpca[:,1]

sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                palette=sns.color_palette("hls", 3),
                data=df).set(title="Image KernelPCA projection")

NameError: ignored

# References

- https://www.microsoft.com/en-us/research/uploads/prod/2006/01/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf
- https://arxiv.org/pdf/1207.3538.pdf
- https://en.wikipedia.org/wiki/Kernel_principal_component_analysis#:~:text=In%20the%20field%20of%20multivariate,a%20reproducing%20kernel%20Hilbert%20space
- https://pages.stat.wisc.edu/~mchung/teaching/MIA/reading/diffusion.gaussian.kernel.pdf
- https://www.mlpack.org/papers/kpca.pdf